In [34]:
import numpy as np
import pandas as pd 
import sklearn

In [35]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [36]:
def crossEntropyLoss(x, t):
    (1 - t) * np.log(sigmoid(x)) + t * np.log(1 - sigmoid(x))

In [37]:
def prob(y):
    return np.array([1 if label == "B" else 0 for label in y])


In [38]:
def miniSGD(batch_size, X, y, w, alpha=0.01, epoch=100):
    m, n = X.shape

    X_b = np.c_[np.ones((m, 1),), X]
    w0 = w

    t = prob(y.values.flatten())

    for i in range(epoch):
        permutation = np.random.permutation(m)
        X_shuffled = X_b[permutation]
        t_shuffled = t[permutation]

        for i in range(0, m, batch_size):
            X_batch = X_shuffled[i:i + batch_size]
            t_batch = t_shuffled[i:i + batch_size]
            y_batch = sigmoid(np.dot(X_batch, w))

            error = y_batch - t_batch
            gradient = np.dot(X_batch.T, error)

            w0 -= alpha * gradient

    return w0

In [39]:
def confusionMatrix(y_pred, y_true):
    TP = np.sum((y_pred == "B") & (y_true == "B"))
    FP = np.sum((y_pred == "B") & (y_true == "M"))
    TN = np.sum((y_pred == "M") & (y_true == "M"))
    FN = np.sum((y_pred == "M") & (y_true == "B"))

    return TP, FP, TN, FN

In [40]:
def eval(y_pred, y_true):
    TP, FP, TN, FN = confusionMatrix(y_pred, y_true)

    accuracy = (TP + TN) / len(y_pred)
    precision = TP / (TP + FP) 
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)

    return accuracy, precision, recall, F1

In [41]:
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split

In [42]:
breast_df = fetch_ucirepo(id=17) 

X = breast_df.data.features 
y = breast_df.data.targets 


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [44]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [45]:
import random

w = np.array([random.gauss(0, 1) for _ in range(X.shape[1] + 1)])

epochs = 100

alpha1 = 0.1 
batch_size1 = 10
w1 = miniSGD(batch_size1, X_train, y_train, w, alpha1, epochs)

alpha2 = 0.01
batch_size2 = 32
w2 = miniSGD(batch_size2, X_train, y_train, w, alpha2, epochs)

alpha3 = 0.001
batch_size3 = 64
w3 = miniSGD(batch_size3, X_train, y_train, w, alpha3, epochs)

alpha4 = 0.0001
batch_size4 = 128
w4 = miniSGD(batch_size4, X_train, y_train, w, alpha4, epochs)

In [46]:
X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]
predicted_probabilities1 = sigmoid(np.dot(X_test_b, w1))
predicted_probabilities2 = sigmoid(np.dot(X_test_b, w2))
predicted_probabilities3 = sigmoid(np.dot(X_test_b, w3))
predicted_probabilities4 = sigmoid(np.dot(X_test_b, w4))

In [47]:
predictions1 = np.where(predicted_probabilities1 > 0.5, "B", "M")
accuracy1, precision1, recall1, F1_1 = eval(predictions1, y_test.values.flatten())
print(f"Accuracy of the model is {accuracy1}")
print(f"Precision of the model is {precision1}")
print(f"Recall of the model is {recall1}")
print(f"F1 of the model is {F1_1} \n")

predictions2 = np.where(predicted_probabilities2 > 0.5, "B", "M")
accuracy2, precision2, recall2, F1_2 = eval(predictions2, y_test.values.flatten())
print(f"Accuracy of the model is {accuracy2}")
print(f"Precision of the model is {precision2}")
print(f"Recall of the model is {recall2}")
print(f"F1 of the model is {F1_2} \n")

predictions3 = np.where(predicted_probabilities3 > 0.5, "B", "M")
accuracy3, precision3, recall3, F1_3 = eval(predictions3, y_test.values.flatten())
print(f"Accuracy of the model is {accuracy3}")
print(f"Precision of the model is {precision3}")
print(f"Recall of the model is {recall3}")
print(f"F1 of the model is {F1_3} \n")

predictions4 = np.where(predicted_probabilities4 > 0.5, "B", "M")
accuracy4, precision4, recall4, F1_4 = eval(predictions4, y_test.values.flatten())
print(f"Accuracy of the model is {accuracy4}")
print(f"Precision of the model is {precision4}")
print(f"Recall of the model is {recall4}")
print(f"F1 of the model is {F1_4} \n")

Accuracy of the model is 0.9627659574468085
Precision of the model is 0.9830508474576272
Recall of the model is 0.9586776859504132
F1 of the model is 0.9707112970711298 

Accuracy of the model is 0.9627659574468085
Precision of the model is 0.9830508474576272
Recall of the model is 0.9586776859504132
F1 of the model is 0.9707112970711298 

Accuracy of the model is 0.9627659574468085
Precision of the model is 0.9830508474576272
Recall of the model is 0.9586776859504132
F1 of the model is 0.9707112970711298 

Accuracy of the model is 0.9627659574468085
Precision of the model is 0.9830508474576272
Recall of the model is 0.9586776859504132
F1 of the model is 0.9707112970711298 



The model is found to be pretty accurate. I am not entirely sure why the metrics do not change as I change the parameters, all I can assume is the changes are too miniscule in my miniSGD function. The change of alpha doesn't effect it much since alpha is already small.